# Project Description
## Xiang Ding
In this project, I have set out on an exciting voyage into the realm of scientific computing, delving into the intricacies of manipulating dataframes, visualizing data through plotting, and leveraging linear regression machine learning algorithms to make predictions and examine the how much impact does each indepent variables factors in the output. To apply these skills, I have meticulously analyzed a dataset containing 1000 samples of students' exam performance, using pandas to see how much the parental education, free school lunch, and exam prep factors in academic success. There are three parts of this project: 
#### Part I: running bootstrap to estimate the true parameters of the exam score based on gender and exam category via data visualization, this will allow me to obtain a more reliable estimate of the means, with a 99% confidence interval approximation. 
#### Part II: Examine on serval factors that may be assoicated with the student exam performance via data visualization.
#### Part III: Is where I am going to walk you step by step of how I use scikit learn to build a linear model and examine the impacts of each independent variables has on the ouputs.
Please uncomment each cell below to run and see the data visualization.

In [ ]:
#!pip install -U scikit-learn
#!pip install pandas
#!pip install numpy
#!pip install -U matplotlib

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from data_module import df_vis
from data_module import data_proc
from data_module import basic_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

### Read the CSV file

In [ ]:
db = pd.read_csv('data/exams.csv')
# Uncomment if you want to see the dataframe
#db.head()

### Part I:
1000 bootstrap iterations for each gender in the exam score category, with a 99% confidence interval approximation.

In [ ]:
# Uncomment the code to see what the plot looks like
#df_vis.bootstrap(db, 'male', 'math score')
#df_vis.bootstrap(db, 'male', 'reading score')
#df_vis.bootstrap(db, 'male', 'writing score')
#df_vis.bootstrap(db, 'female', 'math score')
#df_vis.bootstrap(db, 'female', 'reading score')
#df_vis.bootstrap(db, 'female', 'writing score')

### PartII:
Factors: Parents education level, Free/reduced lunch, test preparation course

In [ ]:
# Uncomment the code to see what the bar graph looks like
#df_vis.parents_education_score(db)
#df_vis.overall_lunch(db)
#df_vis.test_prep(db)

From the analysis of the three factors' data visualizations above, it is evident that there exists a strong correlation between higher academic performance and certain factors. Specifically, students with standard lunch, parents with higher education, and those who have completed exam preparation tend to have higher average exam scores.

### Part III:
I am going to use scikit-learn to build a linear regression model to see how much correlation with students' overall exam performance based on three key factors: standard lunch, parents' education level, and completion of exam preparation. Note: For this project I will only based the overall average exam performance for students.

In the first step, I quantify the data in these categories. To quantify the parents' education background, I assign a numerical value of 3 to indicate a master's degree, 2 for a bachelor's degree, 1 for some college education, and 0 for no college education. As for the second factor, school lunch, we will create a binary variable where a value of 1 indicates that the student receives a free school lunch, and 0 indicates that the student does not receive a free school lunch. Similarly, we will follow the same procedure for exam preparation.

In [ ]:
new_db = data_proc.quantify_df(db).drop(columns=['gender', 'race/ethnicity', 'math score', 'reading score', 'writing score'])

 In the second step, I divide the data into two parts: x_variables and y_variables. x_variables will represent the input variables that will be used in the regression model, while y_variables will represent the target variables.

In [ ]:
x_var = new_db.drop(columns=['overall_score'])

It is important to standardize the overall_score variable to ensure that all variables are on the same scale when building the regression model. The forumla I used is (xi - x.mean)/np.std(x)

In [ ]:
y_var = data_proc.su(new_db.get('overall_score'))

In the third step, I split the data into training and testing subsets using an 80/20 ratio. This means that 80% of the dataset will be used for training the linear regression model, while the remaining 20% will be used for testing. To accomplish this, we will be utilizing the train_test_split function from the sklearn.model_selection library.
##### But feel free to play around the data split percentage to see how the r^2 values changes with different split

In [ ]:
x_train, x_test, y_train, y_test = basic_model.datasplit(x_var, y_var, train=0.8, test=0.2)

Use the data to train the sklearn.linear_model

In [ ]:
model = basic_model.my_model()
model.fit(x_train, y_train)

Apply the trained model to make prediction on the training/test set and output the model stats.

In [ ]:
score_predict_train = basic_model.score_predict_train(model, x_train, y_train)
score_predict_test = basic_model.score_predict_test(model, x_test, y_test)

### Output the equation of the linear model and the model's prediction graph
Pe = Parents education level, l = school lunch, ep = exam prep

In [ ]:
basic_model.linear_eq(model)
df_vis.predict_model(y_train, score_predict_train, y_test)
df_vis.actual(score_predict_test, y_test)

Based on my obervation, the linear model I have are not very accurate in terms of perdicting the student score due to it's high MSE values and low R^2 values. Whic implies that the parents education, school lunch and exam prep does not play a significant roles in the overall exam performance.

In [ ]:
!pytest

## Extra Credits
I belived I have really challenged myself by exploring and utilizing various libraries, including Pandas and Matplotlib, to manipulate data and generate insightful visualizations. However, the most exhilarating part has been diving into the world of machine learning with Scikit-Learn. By delving into the mathematics behind linear regression models and researching various online articles, I've gained a deeper understanding of the mechanics behind the model and how to effectively apply it to my datasets. Moreover, This project has also been uploaded on github as well